In [78]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh


In [1]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE

import pandas as pd


## Notebook overview
goal: check data for duplicates

- All old data is moved into duplicates folder in blobfuse.
- Save new df with duplicates removed, bases on the MB5 hash. 
- Data is re-split into subsets (train, test, val and dev)

In [9]:
txtfiles_tokenizer = pd.read_pickle(f"{cf.output_path}/duplicates/txtfiles_tokenizer.pkl")

In [10]:
import hashlib
import sys

# load function to split data into subsets (train,test,val and dev)
sys.path.append('../scripts/') 
from data_split import save_split

def calculate_md5(file_path):
    """Calculate the MD5 hash of a file."""
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

"""Calculate MD5 hash for each file path in a DataFrame."""
def drop_duplicates(df, file_path_column):
    df['md5_hash'] = df[file_path_column].apply(calculate_md5)

    # remove rows with duplicate md5_hash
    clean_df = df.drop_duplicates(subset=['md5_hash'])
    print(f"{len(df)-len(clean_df)} docs removed. New total: {len(clean_df)} docs.")

    # redo data split
    clean_df = save_split(clean_df)
    return clean_df

# output_file = f"{cf.output_path}/hash_output.txt"
hash_df = drop_duplicates(txtfiles_tokenizer, 'path')


5886 docs removed. New total: 20818 docs.


In [6]:
hash_df.to_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")